In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import urllib, os, csv

# df = pd.read_json(r"C:\Users\mark.ding\big-data-ai-integration-platform\sparkVolumnTest\企業資訊_全_2024_10_24.json")
df = pd.read_json(r"D:\markding_git\big-data-ai-integration-platform\spark_volume\jupyterlab\json_data\企業資訊_for_ml_2025_08.json")

for column in df.columns:
    df[column] = df[column].apply(lambda x : ','.join(map(str,x)) if isinstance(x, list) else x)
for i in df.columns:
    df[i] = df[i].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)


In [ ]:
user = "root"
password = "!QAZ2wsx"
host = "localhost"
port = 3307
db   = "whole_corp"
tbl  = "whole_corp"

encoded_pw = urllib.parse.quote_plus(password)
# IMPORTANT: allow_local_infile=1 for mysqlconnector (or local_infile=1 for PyMySQL)
db_url = f"mysql+mysqlconnector://{user}:{encoded_pw}@{host}:{port}/{db}?allow_local_infile=1"

engine = create_engine(db_url, pool_pre_ping=True)

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS whole_corp (
            統一編號 CHAR(8) NOT NULL,
            公司名稱 VARCHAR(50),
            負責人 VARCHAR(20),
            登記地址 VARCHAR(200),
            資本額 VARCHAR(50),
            營業項目及代碼表 VARCHAR(200),
            縣市名稱 VARCHAR(20),
            區域名稱 VARCHAR(20),
            縣市區域 VARCHAR(20),
            類別_全 VARCHAR(200),
            官網 VARCHAR(100),
            電話 VARCHAR(20),
            PRIMARY KEY (統一編號)
        ) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
    """))
# 1) Ensure table exists with the right schema.
#    If you don't have it yet, create it once with to_sql but with zero rows
#    (or write a CREATE TABLE manually for precise types).
#    Example (create empty table once):
df.head(0).to_sql(tbl, engine, if_exists="append", index=False)

# 2) Write a clean TSV to disk (much safer than CSV for commas/quotes)
tmp_path = "/tmp/whole_corp_load.tsv"  # on Windows: r"C:\temp\whole_corp_load.tsv"
os.makedirs(os.path.dirname(tmp_path), exist_ok=True)

# Write TSV: represent NULLs as \N (MySQL understands \N as NULL if not quoted)
df.to_csv(
    tmp_path,
    sep="\t",
    index=False,
    na_rep="\\N",
    quoting=csv.QUOTE_NONE,
    escapechar="\\",
)

# 3) Bulk load
with engine.begin() as conn:

    # optional but helps: disable checks while loading
    conn.execute(text("SET FOREIGN_KEY_CHECKS=0"))
    conn.execute(text("SET UNIQUE_CHECKS=0"))
    # If you have heavy indexes on the table, consider dropping them before and recreating after.

    # LOAD DATA (LOCAL needs allow_local_infile=1)
    conn.execute(text(f"""
        LOAD DATA LOCAL INFILE :path
        INTO TABLE {tbl}
        FIELDS TERMINATED BY '\t'
        ESCAPED BY '\\\\'
        LINES TERMINATED BY '\n'
        IGNORE 1 LINES
    """), {"path": tmp_path})

    conn.execute(text("SET UNIQUE_CHECKS=1"))
    conn.execute(text("SET FOREIGN_KEY_CHECKS=1"))

In [2]:
from mysql.connector import connect
import pandas as pd
from sqlalchemy import create_engine, text
import urllib, os, csv

# Function to create a MySQL connection
def mysql_connection_whole_corp():
    return connect(host='localhost',
                   port=3307,
                   user='root',
                   password='!QAZ2wsx',
                   database='whole_corp')

con = mysql_connection_whole_corp()
cursor = con.cursor()

cursor.execute('select * from wholecorp_clusters_vector')

result = cursor.fetchall()

df_vector = pd.DataFrame(result, columns= cursor.column_names)

In [4]:
cursor.execute('select * from whole_corp')

result = cursor.fetchall()

df = pd.DataFrame(result, columns= cursor.column_names)


In [9]:
import boto3

In [ ]:
len(df_vector)

1379734

In [8]:
df[~df.統一編號.isin(df_vector.統一編號)]

,統一編號,公司名稱,負責人,登記地址,資本額,營業項目及代碼表,縣市名稱,區域名稱,縣市區域,類別_全,官網,電話
57363,0BO11446,合成商號,\N,屏東縣東港鎮南平里南平路１０２號,\N,日常用品零售業,屏東縣,東港鎮,屏東縣東港鎮,日常用品零售業,\N,\N\r
57364,0WSR0599,MarcobelliBuilder'sCenter,\N,\N,\N,\N,\N,\N,\N,工商專業服務/金融工商/營建設備服務,https://6326710391.web66.com.tw\n,632-6710391\r
1379736,A0342336,臺保實業有限公司,趙O煌,高雄市三民區民孝路６５號３樓,6000000,"五金批發業, 人力派遣業, 國際貿易業, 室內裝潢, 建材批發業, 機械批發業, 門窗安裝工程業",高雄市,三民區,高雄市三民區,"五金批發業, 人力派遣業, 國際貿易業, 室內裝潢, 建材批發業, 機械批發業, 門窗安裝工程業",\N,\N\r
1379737,A9900040,吳麗春商店,\N,宜蘭縣頭城鎮城北里民鋒路９號,\N,"五金批發業, 五金零售業, 日常用品零售業, 餐廳餐館",宜蘭縣,頭城鎮,新竹縣尖石鄉,"五金批發業, 五金零售業, 日常用品零售業, 餐廳餐館",\N,\N\r
1379738,A9900060,連興車行,\N,桃園縣八德市興仁里中山路１６號,\N,美容美髮服務,桃園縣,八德市,澎湖縣七美鄉,美容美髮服務,\N,\N\r
1379739,A9900264,何扶元藥房,\N,新竹縣新埔鎮新埔里一七四號,\N,"寵物食品及其用品批發業, 農產品零售業, 食品什貨、飲料零售業",新竹縣,新埔鎮,新竹縣新埔鎮,"寵物食品及其用品批發業, 農產品零售業, 食品什貨、飲料零售業",\N,\N\r
1379740,A9900506,麗品服飾行,\N,桃園縣大溪鎮一德里中央路３鄰２２之２號,\N,"五金批發業, 五金零售業, 日常用品零售業",桃園縣,大溪鎮,臺南市東區,"五金批發業, 五金零售業, 日常用品零售業",\N,\N\r
1379741,A9900604,文川食品鋪,\N,臺南市中西區永福路172號????????????,\N,"日常用品批發業, 日常用品零售業, 食品什貨、飲料零售業",臺南市,中西區,新竹縣五峰鄉,"日常用品批發業, 日常用品零售業, 食品什貨、飲料零售業",\N,\N\r
1379742,A9900676,裕興號,\N,新竹縣北埔鄉南興村城力街四十六號,\N,"五金批發業, 五金零售業",新竹縣,北埔鄉,臺南市中西區,"五金批發業, 五金零售業",\N,\N\r
1379743,A9901187,昭星商店,\N,新竹縣湖口鄉愛勢村民族街一００號,\N,"五金批發業, 五金零售業, 日常用品零售業",新竹縣,湖口鄉,臺南市東區,"五金批發業, 五金零售業, 日常用品零售業",\N,\N\r
